## Similarity Matrix

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from rdkit.Chem import rdFingerprintGenerator

filename = "biostructures_combined"
path = os.path.join( "data", "datasets", filename + ".csv")
compounds = pd.read_csv(path)
compounds.head()

,smiles
0,C1CCCC(CC1)CCCCCNCCSS(=O)(=O)O
1,CC1=CC2=C(C3=C(CCC3)C(=O)O2)C(=C1)OC4C(C(C(C(O...
2,C1=CC(=CC(=C1)Br)C(=O)NCC(=O)NN=CC2=C(C=C(C=C2...
3,COC(=O)CC(C1=CC=CC=C1)C2=C(C=C(C3=C2OC(=CC3=O)...
4,CC1C(OC(CC1=C)(C(C(=O)NC2C3C(C(C(C(O3)CC(CO)OC...


In [2]:
compounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730464 entries, 0 to 730463
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   smiles  730464 non-null  object
dtypes: object(1)
memory usage: 5.6+ MB


In [3]:
# code taken from https://github.com/florian-huber/molecular_fingerprint_comparisons

import numba
from numba import prange
import numpy as np
from fingerprint_computation import FingerprintGenerator, compute_fingerprints_from_smiles

@numba.njit
def ruzicka_similarity(A, B):
    """
    Calculate the Ruzicka similarity between two count vectors.
    
    Parameters:
    A (array-like): First count vector.
    B (array-like): Second count vector.
    
    Returns:
    float: Ruzicka similarity.
    """
    
    min_sum = np.sum(np.minimum(A, B))
    max_sum = np.sum(np.maximum(A, B))
    
    return min_sum / max_sum


@numba.jit(nopython=True, fastmath=True, parallel=True)
def ruzicka_similarity_matrix(references: np.ndarray) -> np.ndarray:
    """Returns matrix of Ruzicka similarity between all-vs-all vectors of references and queries.

    Parameters
    ----------
    references
        Reference vectors as 2D numpy array. Expects that vector_i corresponds to
        references[i, :].
    queries
        Query vectors as 2D numpy array. Expects that vector_i corresponds to
        queries[i, :].

    Returns
    -------
    scores
        Matrix of all-vs-all similarity scores. scores[i, j] will contain the score
        between the vectors references[i, :] and queries[j, :].
    """

    size = references.shape[0]
    scores = np.zeros((size, size)) #, dtype=np.float32)
    for i in prange(size):
        for j in range(size):
            scores[i, j] = ruzicka_similarity(references[i, :], references[j, :])
    return scores


def compute_similarity_matrix(compounds, sim_matrix_file, morgan_radius=9, fpSize=4096):
    fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=morgan_radius, fpSize=fpSize)
    fingerprints_morgan_count = compute_fingerprints_from_smiles(compounds.smiles, fpgen, count=True, sparse=False, progress_bar=True)
    similarities_morgan_count = ruzicka_similarity_matrix(fingerprints_morgan_count)
    np.save(sim_matrix_file, similarities_morgan_count.astype(np.float32)) # big one ~5GB
    return np.load(sim_matrix_file, mmap_mode ='r')

In [15]:
morgan_radius=9
fpSize=2048
parts = 16
fingerprints_file = os.path.join("data", "group_similarity", f"{filename}_fingerprints_morgan{morgan_radius}_{fpSize}bits.npy")
fingerprints_morgan_count = np.load(fingerprints_file, mmap_mode ='r')
(fingerprints_morgan_count.shape[0] /1500) * fingerprints_morgan_count.shape[0] * 128 * fingerprints_morgan_count.dtype.itemsize / 1024**3

169.6197685546875

In [52]:
bits = 2048
fraction = 8
partsize = 5
print(f"1/{fraction} of the dataset will result in mols = {int(fingerprints_morgan_count.shape[0]/fraction)} mols.")
disk_size = (fingerprints_morgan_count.shape[0]/fraction) * (fingerprints_morgan_count.shape[0]/fraction) * fingerprints_morgan_count.dtype.itemsize / 1024**3
fingerprints_disk_size = (fingerprints_morgan_count.shape[0]/fraction) * bits * fingerprints_morgan_count.dtype.itemsize / 1024**3
print(f"Disk size = {disk_size:.2f} GB. With {partsize} GB per part, we will need {int(disk_size/partsize)} parts.")
print(f"Fingerprints ram size with {bits} = {fingerprints_disk_size:.2f} GB")

1/8 of the dataset will result in mols = 91308 mols.
Disk size = 31.06 GB. With 5 GB per part, we will need 6 parts.
Fingerprints ram size with 2048 = 0.70 GB


In [16]:
fingerprints_morgan_count.shape[0] * fingerprints_morgan_count.shape[1] * fingerprints_morgan_count.dtype.itemsize / 1024**3

5.572998046875

In [5]:
fingerprints_morgan_count.dtype

dtype('float32')

In [ ]:
morgan_radius=9
fpSize=2048
parts = 16

fingerprints_file = os.path.join("data", "group_similarity", f"{filename}_fingerprints_morgan{morgan_radius}_{fpSize}bits.npy")
sim_matrix_files = []
for part in range(parts):
    sim_matrix_files.append(os.path.join("data", "group_similarity", f"{filename}_part{part}_ruzicka_similarities_morgan{morgan_radius}_{fpSize}bits.npy"))
try:
    sim_matrices = [np.load(sim_matrix_file, mmap_mode='r') for sim_matrix_file in sim_matrix_files]
    print(f"Found {len(sim_matrices)} files. Loading similarity matrices.")
except FileNotFoundError:
    print("Sim Matrix Files not found.")
    try:
        fingerprints_morgan_count = np.load(fingerprints_file, mmap_mode ='r')
        print(f"File {fingerprints_file} found. Loading fingerprints.")
    except FileNotFoundError:
        print(f"File {fingerprints_file} not found. Running the fingerprint generation.")
        # fingerprint generation
        fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=morgan_radius, fpSize=fpSize)
        fingerprints_morgan_count = compute_fingerprints_from_smiles(compounds.smiles, fpgen, count=True, sparse=False, progress_bar=True)
        np.save(fingerprints_file, fingerprints_morgan_count.astype(np.float32))
        fingerprints_morgan_count = np.load(fingerprints_file, mmap_mode ='r')
    # sim matrix generation
    print("Fingerprints loaded. Running similarity matrix generation.")
    for part in range(parts):
        sim_matrix_file = sim_matrix_files[part]
        if os.path.exists(sim_matrix_file):
            print(f"File {sim_matrix_file} already exists. Skipping.")
            continue
        print(f"Computing similarity matrix for part {part}.")
        start_idx = int(part * fingerprints_morgan_count.shape[0] / parts)
        end_idx = int((part + 1) * fingerprints_morgan_count.shape[0] / parts)
        %time
        sim_matrix = ruzicka_similarity_matrix(fingerprints_morgan_count[start_idx:end_idx])
        np.save(sim_matrix_file, sim_matrix.astype(np.float32))
        sim_matrix = None
    sim_matrices = [np.load(sim_matrix_file, mmap_mode='r') for sim_matrix_file in sim_matrix_files]

print(sim_matrices[0].shape)
sim_matrices[0]

Sim Matrix Files not found.
File data\group_similarity\biostructures_combined_fingerprints_morgan9_2048bits.npy found. Loading fingerprints.
Fingerprints loaded. Running similarity matrix generation.
Computing similarity matrix for part 0.
CPU times: total: 0 ns
Wall time: 0 ns
Computing similarity matrix for part 1.
CPU times: total: 0 ns
Wall time: 0 ns


## Queries wih similiar groups

#### randomly select 30 analogues, if there are any

In [ ]:
len(sim_matrix)

37811

In [ ]:
sim_matrix[:10]

memmap([[1.        , 0.14537445, 0.06024097, ..., 0.14432989, 0.09448819,
         0.08896797],
        [0.14537445, 1.        , 0.03361345, ..., 0.18604651, 0.07438017,
         0.04727273],
        [0.06024097, 0.03361345, 1.        , ..., 0.04      , 0.10460251,
         0.13618676],
        ...,
        [0.05625   , 0.02731092, 0.11286682, ..., 0.025     , 0.06736842,
         0.11924686],
        [0.06586827, 0.0738255 , 0.01863354, ..., 0.07964602, 0.05952381,
         0.02487562],
        [0.03233831, 0.02319588, 0.06933333, ..., 0.01126761, 0.04010025,
         0.07263923]], dtype=float32)

In [ ]:
import random

def select_analogue_groups(similarity_matrix, group_size=30, sim_range=(0.8, 0.9999), no_overlap=True, seed=42, print_mean_similarity=False):
    """
    Selects groups of similar compounds based on a similarity matrix.
    Each group contains a specified number of compounds that are similar to a given compound within a defined similarity range.
    Parameters: 
        similarity_matrix (np.ndarray): A 2D numpy array where each row represents a compound and each column represents the similarity to other compounds.
        group_size (int): The number of similar compounds to select for each compound.
        sim_range (tuple): A tuple defining the lower and upper bounds of the similarity range to consider for selecting similar compounds.
        no_overlap (bool): If True, ensures that selected compounds do not overlap with previously selected compounds.
        seed (int): Random seed for reproducibility.
        print_mean_similarity (bool): If True, prints the mean similarity of the selected compounds for each compound.
        Returns:
            analogue_dict (dict): A dictionary where keys are indices of the original compounds and values are lists of indices of selected similar compounds.
    """
    random.seed(seed)
    analogue_dict = {}
    used_indices =[]
    for i in range(len(similarity_matrix)):
        similar_indices = np.where((similarity_matrix[i] >= sim_range[0]) & (similarity_matrix[i] <= sim_range[1]))[0]
        # remove already used ids from similar_indices
        similar_indices = [idx for idx in similar_indices if idx not in used_indices]
        # check if group size is large enough
        if len(similar_indices) >= group_size:
            if print_mean_similarity:
                mean_similarity = np.mean(similarity_matrix[i][similar_indices])
                print(f"Index {i}: Found {len(similar_indices)} similar compounds with mean similarity {mean_similarity:.3f}. Picking {group_size} random matches.")
            
            random_matches = random.sample(list(similar_indices), group_size)
            #random_matches.sort()
            analogue_dict[i] = random_matches
            if no_overlap:
                used_indices.append(i)
                used_indices.extend(random_matches)
    
    return analogue_dict

In [ ]:
analogue_dict = select_analogue_groups(sim_matrix, group_size=30, sim_range=(0.8, 0.9999), no_overlap=True, seed=42)#, print_mean_similarity=True)
print(len(analogue_dict))
#analogue_dict

5


{12: [36118,
  4964,
  1846,
  15575,
  13371,
  31051,
  6191,
  21350,
  3566,
  16668,
  26415,
  33912,
  25711,
  9389,
  35444],
 809: [2119,
  5949,
  20237,
  20602,
  23223,
  25548,
  1578,
  25294,
  7248,
  27739,
  25745,
  28949,
  32941,
  35992,
  9181],
 1578: [34429,
  13876,
  2274,
  28197,
  37231,
  3216,
  28047,
  6317,
  5671,
  4893,
  33154,
  2936,
  11227,
  35699,
  36397],
 3113: [26266,
  14563,
  25409,
  34396,
  19814,
  14897,
  37106,
  3193,
  18801,
  19710,
  3971,
  16334,
  29081,
  28459,
  12810],
 7630: [34874,
  30652,
  29799,
  18830,
  34437,
  11144,
  3761,
  1790,
  30883,
  27423,
  7320,
  25558,
  36876,
  14336,
  23933]}

In [ ]:
sim_matrix[9][27959]

0.72477067